ref: https://www.kaggle.com/hamishdickson/bidirectional-lstm-in-keras-with-glove-embeddings

# LOAD libs

In [1]:
import pandas as pd
import numpy as np
import time
import os, gc

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import *
from keras.utils.np_utils import to_categorical
from keras.initializers import Constant
import re

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
%%time
train_df=pd.read_csv("../KB_NLP/morphs/komo_morphs_train.csv")
test_df=pd.read_csv("../KB_NLP/morphs/komo_morphs_test.csv")

CPU times: user 2.02 s, sys: 102 ms, total: 2.13 s
Wall time: 2.13 s


In [3]:
train_df.head()

,id,year_month,text,smishing,komo_morphs
0,0,2017-01,XXX은행성산XXX팀장입니다.행복한주말되세요,0,XXX 은행 성산 XXX 팀장 이 ㅂ니다 . 행복 한 주말 되 시 어요
1,1,2017-01,오늘도많이웃으시는하루시작하세요XXX은행 진월동VIP라운지 XXX올림,0,오늘 도 많이 웃 으시 는 하루 시작 하 시 어요 XXX 은행 지 ㄴ 월동 VIP ...
2,2,2017-01,안녕하십니까 고객님. XXX은행입니다.금일 납부하셔야 할 금액은 153600원 입니...,0,안녕하십니까 고객 님 . XXX 은행 이 ㅂ니다 . 금일 납부 하 시 어야 하 ㄹ ...
3,4,2017-01,XXX 고객님안녕하세요XXX은행 XXX지점입니다지난 한 해 동안 저희 XXX지점에 ...,0,XXX 고객 님 안녕하세요 XXX 은행 XXX 지점 이 ㅂ니다 지나 ㄴ 한 하 아 ...
4,5,2017-01,1월은 새로움이 가득XXX입니다.올 한해 더 많이행복한 한해되시길바랍니다,0,1월 은 새롭 ㅁ 이 가득 XXX 이 ㅂ니다 . 올 한하 아 더 많이 행복 하 ㄴ ...


In [4]:
pd.set_option('display.max_colwidth',-1)
train_df.head(20)

,id,year_month,text,smishing,komo_morphs
0,0,2017-01,XXX은행성산XXX팀장입니다.행복한주말되세요,0,XXX 은행 성산 XXX 팀장 이 ㅂ니다 . 행복 한 주말 되 시 어요
1,1,2017-01,오늘도많이웃으시는하루시작하세요XXX은행 진월동VIP라운지 XXX올림,0,오늘 도 많이 웃 으시 는 하루 시작 하 시 어요 XXX 은행 지 ㄴ 월동 VIP 라운지 XXX 올림
2,2,2017-01,안녕하십니까 고객님. XXX은행입니다.금일 납부하셔야 할 금액은 153600원 입니다.감사합니다. 새해 복 많이 받으십시오.XXX은행옥포XXX올림,0,안녕하십니까 고객 님 . XXX 은행 이 ㅂ니다 . 금일 납부 하 시 어야 하 ㄹ 금액 은 153600 원 이 ㅂ니다 . 감사 하 ㅂ니다 . 새해 복 많이 받 으시 ㅂ시오 . XXX 은행 옥포 XXX 올림
3,4,2017-01,XXX 고객님안녕하세요XXX은행 XXX지점입니다지난 한 해 동안 저희 XXX지점에 보내주신 성원에 감사드립니다. 설렘으로 시작한 2017년소망하시는 일 모두 이XXX 고객님의 가정에 늘 건강과 행복이 함께하길 기원하겠습니다. 사랑하는 가족과 함께 정을 나누는 행복한 설 명절 보내세요 XXX은행 XXX지점직원일동,0,XXX 고객 님 안녕하세요 XXX 은행 XXX 지점 이 ㅂ니다 지나 ㄴ 한 하 아 동안 저희 XXX 지점 에 보내 어 주 시 ㄴ 성원 에 감사 드리 ㅂ니다 . 설레 ㅁ 으로 시작 하 ㄴ 2017년 소망 하 시 는 일 모두 이 XXX 고객 님 의 가정 에 늘 건강 과 행복 이 함께 하 기 ㄹ 기원 하 겠 습니다 . 사랑 하 는 가족 과 함께 정 을 나누 는 행복 하 ㄴ 설 명절 보내 시 어요 XXX 은행 XXX 지점 직원 일동
4,5,2017-01,1월은 새로움이 가득XXX입니다.올 한해 더 많이행복한 한해되시길바랍니다,0,1월 은 새롭 ㅁ 이 가득 XXX 이 ㅂ니다 . 올 한하 아 더 많이 행복 하 ㄴ 한해 되 시 기 ㄹ 바라 ㅂ니다
5,6,2017-01,행복한주말보내세요XXX용현남전담직원대리 XXX올림,0,행복 하 ㄴ 주말 보 내세 요 XXX 용 현 남 전담 직원 대리 XXX 올림
6,7,2017-01,XXX 고객님 안녕하세요XXX은행 무교지점 XXX과장입니다 오늘 아침에 눈을 뜨니 눈이 많이 와서 온 세상이 하얗게 변했더군요어릴적에는 눈이 마냥 좋기만 했는데눈을 보는 순간 좋기보다는 먼저. 출근을 걱정하게 되더라구요이제 진정한 어른이 되었음을 느끼게 됩니다새해 인사를 드리기전에..제가 발령이 나서 본점으로 가게되었어요ㅠㅠ. 진정한 구정 새해인사보다 이별인사를 먼저 드리게 되어 송구할 뿐입니다그동안 보내주신 성원에 감사.또 감사드리며XXX은행에 있는XXX고객님을 평생 저의 고객으로 모시겠습니다.새해 복 많이 받으시고 2017년 원하고 바라시는 모든일 이루시길 기원합니다XXX은행 무교지점 XXX과장올림,0,XXX 고객 님 안녕하세요 XXX 은행 무교 지점 XXX 과장 이 ㅂ니다 오늘 아침 에 눈 을 뜨 니 눈 이 많이 오 아서 오 ㄴ 세상 이 하얗 게 변하 았 더군요 어리 ㄹ 적 에 는 눈 이 마냥 좋 기 만 하 았 는데 눈 을 보 는 순간 좋 기 보다 는 먼저 . 출근 을 걱정 하 게 되 더라구요 이제 진정 하 ㄴ 어른 이 되 었 음 을 느끼 게 되 ㅂ니다 새해 인사 를 드리 기 전 에 .. 제가 발령 이 나서 어 본점 으로 가게되었어요ㅠㅠ. 진정 하 ㄴ 구정 새 해인사 보다 이별 인사 를 먼저 드리 게 되 어 송구 하 ㄹ 뿐 이 ㅂ니다 그동안 보내 어 주 시 ㄴ 성원 에 감사 . 또 감사 드리 며 XXX 은행 에 있 는 XXX 고객 님 을 평생 저 의 고객 으로 모시 겠 습니다 . 새해 복 많이 받 으시 고 2017 년 원하 고 바라 시 는 모든 일 이 루시 길 기원 하 ㅂ니다 XXX 은행 무교 지점 XXX 과 장 올림
7,8,2017-01,XXX 고객님지난 한해 베풀어 주신 은혜 진심으로 감사 드립니다.가슴 깊이 간직 하겠습니다.정유년 새해 가족 모두 행복하시고 뜻 하는 바 모두이루시길 진심으로 소망합니다XXX XXX은행 XXX 지점장 정 XXX 배상,0,XXX 고객 님 지나 ㄴ 한하 아 베풀 어 주 시 ㄴ 은혜 진심 으로 감사 드리 ㅂ니다 . 가슴 깊이 간직 하 겠 습니다 . 정유 년 새해 가족 모두 행복 하 시 고 뜻 하 는 바 모두 이루 시 기 ㄹ 진심 으로 소망 하 ㅂ니다 XXX XXX 은행 XXX 지점장 정 XXX 배상
8,9,2017-01,설연휴 가족들과 훈훈한 정 나누시고 정겨운추억 많이 만드세요XXX오XXX올림,0,설 연휴 가족 들 과 훈훈 하 ㄴ 정 나누 시 고 정겨운 추억 많이 만들 시 어요 XXX 오 XXX 올림
9,10,2017-01,(광고)XXXBaXXX고객님들 뒤엔XXX 언제나 XXX새로운 마음가짐으로 새롭게 준비합니다.당행상품의 자격기준과 심사기준이 완화되어 상품에 대해 간단하게 상품정보 전달드립니다.수신을 희망하지 않으실 경우에는 거부 라는 답장을 주시면 KISA 수신거부 목록에 등록을 시켜 두번 다시 발송되지 않도록 조치를 취해드리겠습니다.신청방법: XXX-XXX-XXX 터치하셔서 문자로 상담 또는 00시 00분 상담이라고 답장을 보내주시거나 전화를 주시면 친절하고 안전한 상담으로 도움드리겠습니다.카카오톡 상담서비스 시행중 citibankloan친구추가 후 상담신청이런 분들께 권해드립니다.1. 시중은행권에서 기존한도를 모두 사용 중이신 분.2. 기대출의 원리금 균등방식으로 인해 월 불입금액이 부담스러우신 분.3. 총 채무금액은 작으나 채무건수가 많아서 관리가 안 되시는 XXX고객님들께 말씀드립니다. 높은 금리 때문에 힘들다고 느끼셨을 때 대출이 필요하실 때 언제나XXX 고객님들 뒤에서 든든한 파트너로서 도와드리겠습니다.당행 상품특징1. 당행 거래가 없으셔도 진행가능.2. 시중은행권보다 높은 한도.3. 자체 등급으로 판단하므로 낮은신용자도 진행가능.4. 부채가 많다면 부채 통합상품으로 전환가능.5. 원리금 균등방식을 이자만 납입하는 만기일시로 전환가능.6. 가상 조회를 통해 한도 및 금리를 한 번에 확인가능.당행 상품안내1. 직장인전용 신용상품한도: 최대 1억 4000만 원까지.금리: 최저 2.97%자격: XXX은행 자체 등급과 기업 리스트에 따라 차등적용.2. 채무통합 전환상품한도: 연봉 대비 300%까지.금리: 최저 2.97%자격: XXX은행 자체 등급과 기업 리스트에 따라 차등적용.3. 새 희망 홀씨한도: 최대 2500만 원.금리: 8.68%12%자격: 연봉 4000만 원 이하 고객 대상으로 XXX 자체 등급에 따라(광고)XXX,1,( 광고 ) XXXBaXXX 고객 님 들 뒤 엔 XXX 언제나 XXX 새롭 ㄴ 마음가짐 으로 새롭 게 준비 하 ㅂ니다 . 당행 상품 의 자격 기준 과 심사 기준 이 완화 되 어 상품 에 대하 아 간단 하 게 상품 정보 전달 드리 ㅂ니다 . 수신 을 희망 하 지 않 으시 ㄹ 경우 에 는 거부 이 라는 답장 을 주 시 면 KISA 수신 거부 목록 에 등록 을 시키 어 두 번 다시 발송 되 지 않 도록 조치 를 취하 아 드리 겠 습니다 . 신청 방법 : XXX - XXX - XXX 터치 하 시 어서 문자 로 상담 또는 00 시 00 분 상담 이 라고 답장 을 보내 어 주 시 거나 전화 를 주 시 면 친절 하 고 안전 하 ㄴ 상담 으로 도움 드리 겠 습니다 . 카카오톡 상담 서비스 시행 중 citibankloan 친구 추가 후 상담 신청 이런 분 들 께 권하 아 드리 ㅂ니다 . 1 . 시중 은행 권 에서 기존 한도 를 모두 사용 중 이신 분 . 2 . 기 대출 의 원리금 균등 방식 으로 인해 월 불 입금액 이 부담 스럽 시 ㄴ 분 . 3 . 총 채무 금액 은 작 으나 채무 건수 가 많 아서 관리 가 안 되 시 는 XXX 고객 님 들 께 말씀 드리 ㅂ니다 . 높 은 금리 때문 에 힘들 다고 느끼 시 었 을 때 대출 이 필요 하 시 ㄹ 때 언제 나 XXX 고객 님 들 뒤 에서 든든 하 ㄴ 파트너 로서 돕 아 드리 겠 습니다 . 당행 상품 특징 1 . 당행 거래 가 없으셔도 진행 가능 . 2 . 시중 은행 권 보다 높 은 한도 . 3 . 자체 등급 으로 판단 하 므

In [5]:
train_df['l'] = train_df['komo_morphs'].apply(lambda x: len(str(x).split(' ')))
print('mean length of data : {}'.format(train_df['l'].mean()))
print('max length of data : {}'.format(train_df['l'].max()))
print('std length of data : {}'.format(train_df['l'].std()))

mean length of data : 89.27805504401155
max length of data : 644
std length of data : 114.02346925047749


In [6]:
test_df['l'] = test_df['komo_morphs'].apply(lambda x: len(str(x).split(' ')))
print('mean length of data : {}'.format(test_df['l'].mean()))
print('max length of data : {}'.format(test_df['l'].max()))
print('std length of data : {}'.format(test_df['l'].std()))

mean length of data : 152.6691266912669
max length of data : 565
std length of data : 116.83732804508064


---
### Sampling for Debug

In [7]:
df_0 = train_df[train_df['smishing']==0].sample(frac=1)
df_1 = train_df[train_df['smishing']==1].sample(frac=1)

# balancing the training
sample_size = min(len(df_0),len(df_1))

data = pd.concat([df_0.head(sample_size), df_1.head(sample_size)]).sample(frac=1)

In [8]:
data.shape

(37406, 6)

In [9]:
data.head()

,id,year_month,text,smishing,komo_morphs,l
66536,77683,2017-05,항상 거래감사드리며 즐거운 주말 맞으시길 바랍니다XXX방촌 XXX 계장,0,항상 거래 감사 드리 며 즐겁 ㄴ 주말 맞 으시 기 ㄹ 바라 ㅂ니다 XXX 방촌 XXX 계장,18
114157,132637,2017-06,XXX 고객XXX 고객님의 업무를 도와드린 XXX 대리입니다. 항상 XXX은행 모란역 지점을 이용해주셔서 감사드립니다. 추가문의 사항이 있으시면 이번호로 전화주세요 성심껏 도와드리겠습니다. 행복한 저녁시간 되시고 다음에 또 뵙겠습니다. XXX은행모란역XXX올림XXX은행모란역XXX올림,0,XXX 고객 XXX 고객 님 의 업무 를 돕 아 드리 ㄴ XXX 대리 이 ㅂ니다 . 항상 XXX 은행 모란역 지점 을 이용 하 아 주 시 어서 감사 드리 ㅂ니다 . 추가 문 의 사항 이 있 으시 면 이번 호로 전화 주 시 어요 성심껏 돕 아 드리 겠 습니다 . 행복 하 ㄴ 저녁 시간 되 시 고 다음 에 또 뵙 겠 습니다 . XXX 은행 모란역 XXX 올림 XXX 은행 모란역 XXX 올림,79
176754,202300,2017-11,XXX 고객님XXX은행 XXX지점을 이용해 주셔서 감사드립니다. 업무 처리 하시면서 궁금하시거나 불편하신 점은 없으셨XXX 고객님께서 항상 매우 만족 하실 수 있도록 최선을 다하겠습니다 오늘도 즐겁고 기쁜시간 매우 만족하셨길 바라며 남은 저녁 시간 행복한하루 마무리 하시길 바래요 oXXX은행김포한강XXX올림,0,XXX 고객 님 XXX 은행 XXX 지점 을 이용 하 아 주 시 어서 감사 드리 ㅂ니다 . 업무 처리 하 시 면서 궁금 하 시 거나 불편 하 시 ㄴ 점 은 없으셨XXX 고객 님 께서 항상 매우 만족 하 시 ㄹ 수 있 도록 최선 을 다 하 겠 습니다 오늘 도 즐겁 고 기쁘 ㄴ 시간 매우 만족 하 시 었 기 ㄹ 바라 며 남은 저녁 시간 행복 하 ㄴ 하루 마무리 하 시 기 ㄹ 바래 요 oXXX 은행 김포 한강 XXX 올림,88
131572,151601,2017-08,안녕하세요바쁜XXX 지장드릴까봐 문자로 연락 드립니다.한국 XXX 은행 에서 조회및실행업무를맡고있는 담당입니다. 현재 다른은행상품을 보유중추가자금 필요 은행에서 한도발생되지 않으면 아래내용을 읽어봐 주시길 바랍니다.매월상환금이 부담고금리를 이용중이라면 자신있게 권해드립니다고금리 및 다른금융을 XXX은행으로 대환 및 통합해드립니다.현재 은행권에서 가장 완화된 조건입니다.1.직장인전문직대상2.최저 43.1억4천까지월급의 최고 21배까지4.연봉대비 부채가 높아도 가능원리금방식을 이자만 내는 만기일시 가능.6.전화문자 주시면 친절상담해 드립니다조회는 신용등급에 영향이 없습니다.가능여부를 확인하시고 서류 접수는 저희가 계신지역으로 찾아뵙는 방문시스템 입니다.XXX고객님께 정직힘이되는 XXX은행이 되도록 하겠습니다.위문자는 당행과상담상품서비스 안내등에 동의하셨던 분들을 대상으로 보내고 있습니다.혹시 관련이 없으실 경우번호오타로 인한 오류 입니다.잘못된 수신이나수신을 원하지 않으실 경우 거부 문자 남겨주시면 조치 하겠습니다안녕하세요바쁜XXX,1,안녕하세요 바쁘 ㄴ XXX 지장 드리 ㄹ까 보 아 문자 로 연락 드리 ㅂ니다 . 한국 XXX 은행 에서 조회 및 실행 업무 를 맡 고 있 는 담당 이 ㅂ니다 . 현재 다른 은행 상품 을 보유 중 추가 자금 필요 은행 에서 한도 발생 되 지 않 으면 아래 내용 을 읽 어 보 아 주시 길 바라 ㅂ니다 . 매월 상환 금 이 부담 고금리 를 이용 중 이 라면 자신 있 게 권하 아 드리 ㅂ니다고 금리 및 다른 금융 을 XXX 은행 으로 대화 ㄴ 및 통합 하 아 드리 ㅂ니다 . 현재 은행 권 에서 가장 완화 되 ㄴ 조건 이 ㅂ니다 . 1 . 직장인 전문직 대상 2 . 최저 43 . 1 억 4 천 까지 월급 의 최고 21 배 까지 4 . 연봉 대비 부채 가 높 아도 가능 원리금 방식 을 이자 만 내 는 만기일 시 가능 . 6 . 전화 문자 주 시 면 친절 상담 하 아 드리 ㅂ니다 조회 는 신용등급 에 영향 이 없 습니다 . 가능 여부 를 확인 하 시 고 서류 접수 는 저희 가 계시 ㄴ 지역 으로 찾아뵙 는 방문 시스템 이 ㅂ니다 . XXX 고객 님 께 정직 힘 이 되 는 XXX 은행 이 되 도록 하 겠 습니다 . 위문 자 는 당행 과 상담 상품 서비스 안내 등에 동의 하 시 었 던 분 들 을 대상 으로 보내 고 있 습니다 . 혹시 관련 이 없 으시 ㄹ 경우 번호 오타 로 인하 ㄴ 오류 이 ㅂ니다 . 잘못 되 ㄴ 수신 이나 수신 을 원하 지 않 으시 ㄹ 경우 거부 문자 남기 어 주 시 면 조치 하 겠 습 니다 안녕하세요 바쁘 ㄴ XXX,283
179274,205117,2017-11,겨울나무에핀눈꽃이예쁜날.작은기쁨이모여큰행복이되는주말보내세요XXX상암XXX,0,겨울 나무 에피 ㄴ 눈꽃 이 예쁘 ㄴ 날 . 작 은 기쁨 이모 이 어 크 ㄴ 행복 이 되 는 주말 보 내세 요 XXX 상 암 XXX,30


In [10]:
data['l'] = data['komo_morphs'].apply(lambda x: len(str(x).split(' ')))
print('mean length of data : {}'.format(data['l'].mean()))
print('max length of data : {}'.format(data['l'].max()))
print('std length of data : {}'.format(data['l'].std()))

mean length of data : 241.93859273913276
max length of data : 637
std length of data : 199.25565859691557


In [11]:
sequence_length = 52 #220 # Heuristic way to select this value (if possible, max length bewteen train and test would be fine, not considering the resources.)

In [12]:
%%time
max_features = 20000 # oov max range if the number of oov is below this value, then max_feature is set to that value.

tokenizer = Tokenizer(num_words = max_features, split=' ', oov_token='<unw>', filters=' ')
tokenizer.fit_on_texts(data['komo_morphs'].values)

#this takes our sentences and replaces each word with an integer
X = tokenizer.texts_to_sequences(data['komo_morphs'].values)
X = pad_sequences(X, sequence_length) #padding empty to zeros
y = data['smishing']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

print("test set size : {}".format(len(X_test)))

test set size : 3741
CPU times: user 7.14 s, sys: 16 ms, total: 7.16 s
Wall time: 7.16 s


In [13]:
X[:5]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,  208,  150,   99,   14,  205,  514,    9,  365,  833,   65,
          82,   42,  146,    7,    5, 8984,    5,  645],
       [   6,   98,   99,   14,    7,    2,   81,  489,   27,  604,    4,
          32,   65,   43,  512, 2417,   70,   33,    6,   75, 1531,  281,
          13,   14,   48,   28,    2,  192,    3,    9,  840,  137,   22,
           6,   34, 1035,   20, 1056, 1127,   48,   28,    2,    5,   36,
        3309,    5,  140,    5,   36, 3309,    5,  140],
       [ 435,  208,  330,  508,    3,    6,   42,   60,   32,  168,  394,
           8,  226,    3,   48,   28,  350,   59,  514,   34, 1332,    9,
         137,  330,  508,    3,    6,  126,   82,   42,  146,  205, 1395,
         840,  137,  192,    3,    9,  284,  814,    3,    6,   82,   42

In [14]:
# as we set sequence_length = 52, X has 637 values for each data
X.shape

(37406, 52)

In [15]:
word_index = tokenizer.word_index
print('Found {} unique tokens'.format(len(word_index)))

Found 19118 unique tokens


In [16]:
embeddings_index = {}
f = open(os.path.join('../KB_NLP','glove_5290.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:],dtype='float32')
    embeddings_index[word] = coefs
f.close()

print("Found {} word vectors".format(len(embeddings_index)))

Found 5290 word vectors


In [17]:
num_words = min(max_features, len(word_index))+1
print(num_words)

embedding_dim = 200

#first create a matrix of zeros, this is our embedding matrix
embedding_matrix = np.zeros((num_words, embedding_dim))

for word, i in word_index.items():
    if i > max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
    else:
        embedding_matrix[i] = np.random.randn(embedding_dim)

19119


In [18]:
model = Sequential()
model.add(Embedding(num_words,
                   embedding_dim,
                   embeddings_initializer=Constant(embedding_matrix),
                   input_length=sequence_length,
                   trainable=True))
model.add(SpatialDropout1D(0.2))
model.add(Bidirectional(CuDNNLSTM(64, return_sequences=True)))
model.add(Bidirectional(CuDNNLSTM(32)))
model.add(Dropout(0.25))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 52, 200)           3823800   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 52, 200)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 52, 128)           136192    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 64)                41472     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 4,001,529
Tr

In [19]:
import tensorflow as tf
tf.test.is_gpu_available()

False

In [20]:
batch_size = 64
history = model.fit(X_train, y_train, epochs=5, batch_size=batch_size, verbose=1, validation_split=0.1)

InvalidArgumentError: No OpKernel was registered to support Op 'CudnnRNN' used by node bidirectional_1/CudnnRNN (defined at /usr/local/lib/python3.6/dist-packages/keras/layers/cudnn_recurrent.py:517) with these attrs: [seed=87654321, dropout=0, T=DT_FLOAT, input_mode="linear_input", direction="unidirectional", rnn_mode="lstm", is_training=true, seed2=0]
Registered devices: [CPU, XLA_CPU, XLA_GPU]
Registered kernels:
  device='GPU'; T in [DT_DOUBLE]
  device='GPU'; T in [DT_FLOAT]
  device='GPU'; T in [DT_HALF]

	 [[bidirectional_1/CudnnRNN]]

Errors may have originated from an input operation.
Input Source operations connected to node bidirectional_1/CudnnRNN:
 bidirectional_1/concat (defined at /usr/local/lib/python3.6/dist-packages/keras/layers/cudnn_recurrent.py:60)	
 bidirectional_1/transpose (defined at /usr/local/lib/python3.6/dist-packages/keras/layers/cudnn_recurrent.py:484)	
 bidirectional_1/ExpandDims_1 (defined at /usr/local/lib/python3.6/dist-packages/keras/layers/cudnn_rec